<a href="https://colab.research.google.com/github/RabiyaAkhtar/FlaskWeatherPredictionAPI/blob/main/WeatherTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyngrok

In [4]:
# Step 1: Running Flask app in background
from flask import Flask, request, jsonify
from threading import Thread
import requests
from getpass import getpass
from pyngrok import ngrok

# Flask app setup
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict_weather():
    data = request.get_json()

    lat = data.get('lat')
    lon = data.get('lon')
    api_key = data.get('api_key')

    if not all([lat, lon, api_key]):
        return jsonify({'error': 'Missing data'}), 400

    try:
        url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}&units=metric"
        response = requests.get(url)
        weather_data = response.json()

        if response.status_code != 200:
            return jsonify({'error': weather_data.get('message', 'API error')}), response.status_code

        description = weather_data['weather'][0]['description']
        temp = weather_data['main']['temp']
        city = weather_data.get('name', 'Unknown location')
        prediction = f"Weather in {city}: {description}, {temp}°C"

        return jsonify({'prediction': prediction})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

def run_app():
    app.run(port=5000)

In [ ]:
# Step 2: Starting Flask in a thread
Thread(target=run_app).start()

In [ ]:
# Step 3: Ngrok authentication and tunnel setup
auth_token = getpass("Enter your ngrok auth token: ")
ngrok.set_auth_token(auth_token)
public_url = ngrok.connect(5000)
print(" * ngrok tunnel URL:", public_url)

In [ ]:
# Step 4: Sending POST request automatically using the latest ngrok URL
import time
import requests

time.sleep(2)  # Allowing server to fully spin up

# Taking user input for latitude and longitude
lat = float(input("Enter latitude: "))
lon = float(input("Enter longitude: "))

data = {
    "lat": lat,
    "lon": lon,
    'api_key': '2dcee931adcff8d466293f94112eadad'
}


response = requests.post(f"{public_url.public_url}/predict", json=data)

print("Status Code:", response.status_code)
if response.status_code == 200:
    print("Response JSON:", response.json())
else:
    print("Error:", response.text)
